## Shardul Nazirkar
Github - shardul1704

## Multiclass Multilabel Classification Anuran Calls MFCCs Data Set
https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29

### Importing necessary libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, hamming_loss, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
from imblearn.pipeline import Pipeline as imbpipeline
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

### 1. (a) Importing the dataset

In [3]:
df = pd.read_csv("../Data/Frogs_MFCCs.csv")
df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1.0
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1.0
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1.0
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1.0
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,1.0,0.188331,0.129170,0.555004,0.254651,-0.049270,-0.113402,0.109602,0.212008,-0.037427,...,0.082667,-0.050679,-0.086233,-0.014260,0.128203,0.095290,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,18.0
2366,1.0,0.151378,0.092607,0.560406,0.253681,-0.067382,-0.155365,0.138661,0.279432,0.023142,...,0.077878,-0.056092,-0.054308,-0.009268,0.089783,0.066903,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,18.0
2367,1.0,0.124058,0.114009,0.557505,0.232406,-0.067412,-0.130254,0.095365,0.251530,-0.018291,...,0.062188,-0.060865,-0.065491,0.003033,0.092174,0.056448,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,18.0
2368,1.0,0.113697,0.069683,0.531637,0.201875,-0.107971,-0.186037,0.120668,0.292621,-0.004164,...,0.106466,-0.082228,-0.116439,-0.015736,0.181898,0.106481,Leptodactylidae,Adenomera,AdenomeraHylaedactylus,18.0


In [4]:
X = df.iloc[:,:22]
y = df[['Family','Genus','Species']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=32)

### 1. (b) i. Exact match and Hamming loss

##### Exact Match Ratio

Exact Match ratio (subset accuracy) ignores partially correct (considers them incorrect) and extends the accuracy used in single label case for multi-label prediction.

Exact Match Ratio, MR = 1/n ($(\sum \limits _{i=1} ^{n} I(y_{i} = \hat{y}_{i})$)

where I
is the indicator function. Clearly, a disadvantage of this measure is that it does not distinguish between complete incorrect and partially correct which might be considered harsh.

There is a function in scikit-learn which implements subset accuracy, called as accuracy_score.

##### Hamming Loss
It reports how many times on average, the relevance of an example to a class label is incorrectly predicted. Therefore, hamming loss takes into account the prediction error (an incorrect label is predicted) and missing error (a relevant label not predicted), normalized over total number of classes and total number of examples.

Hamming Loss=1nL∑i=1n∑j=1LI(yji≠y^ji)
Hamming Loss = 1/nL ($\sum \limits _{i=1} ^{n} \sum \limits _{j=1} ^{L} I(y_{i}^{j} = \hat{y}_{i}^{j})$)

where I
is the indicator function. Ideally, we would expect the hamming loss to be 0, which would imply no error; practically the smaller the value of hamming loss, the better the performance of the learning algorithm.

### 1. (b) ii. SVM for each of the labels, using Gaussian kernels and one versus all classifiers

In [20]:
c = [1e-3, 1e-2, 1e-1, 1,1e+1,1e+2,1e+3,1e+4]
gamma = np.linspace(0.1, 2, 20)
param_grid = {'C':c,'gamma':gamma}
modelSVM = SVC(kernel='rbf',decision_function_shape='ovr')
gridSearch = GridSearchCV(modelSVM, param_grid=param_grid, cv=10)
FamilyModel=gridSearch.fit(X_train,y_train['Family'])
BestCFamily = FamilyModel.best_params_['C']
BestGammaFamily = FamilyModel.best_params_['gamma']
BestModelFamilyOVR = SVC(C = BestCFamily, kernel = 'rbf', gamma = BestGammaFamily,decision_function_shape='ovr')
FamilyFinalModel = BestModelFamilyOVR.fit(X_train,y_train['Family'])
y_pred = FamilyFinalModel.predict(X_test)
HammingLossFamily = hamming_loss(list(y_test['Family']), y_pred)
ExactMatchFamily = accuracy_score(list(y_test['Family']), y_pred)
print('For the Family label : Best value of C =',BestCFamily)
print('For the Family label : Best value of gamma =',BestGammaFamily)
print('For the Family label : Hamming loss =',HammingLossFamily)
print('For the Family label : Exact Match =',ExactMatchFamily)

For the Family label : Best value of C = 1
For the Family label : Best value of gamma = 1.4
For the Family label : Hamming loss = 0.0028129395218002813
For the Family label : Exact Match = 0.9971870604781997


In [21]:
modelSVM = SVC(kernel='rbf',decision_function_shape='ovr')
gridSearch = GridSearchCV(modelSVM, param_grid=param_grid, cv=10)
GenusModel = gridSearch.fit(X_train,y_train['Genus'])
BestCGenus = GenusModel.best_params_['C']
BestGammaGenus = GenusModel.best_params_['gamma']
BestModelGenusOVR = SVC(C = BestCGenus, kernel = 'rbf', gamma = BestGammaGenus,decision_function_shape='ovr')
GenusFinalModel = BestModelGenusOVR.fit(X_train,y_train['Genus'])
y_pred = GenusFinalModel.predict(X_test)
HammingLossGenus = hamming_loss(list(y_test['Genus']), y_pred)
ExactMatchGenus = accuracy_score(list(y_test['Genus']), y_pred)
print('For the Genus label : Best value of C =',BestCGenus)
print('For the Genus label : Best value of gamma =',BestGammaGenus)
print('For the Genus label : Hamming loss =',HammingLossGenus)
print('For the Genus label : Exact Match =',ExactMatchGenus)

For the Genus label : Best value of C = 1
For the Genus label : Best value of gamma = 1.4
For the Genus label : Hamming loss = 0.0028129395218002813
For the Genus label : Exact Match = 0.9971870604781997


In [24]:
modelSVM = SVC(kernel='rbf',decision_function_shape='ovr')
gridSearch = GridSearchCV(modelSVM, param_grid=param_grid, cv=10)
SpeciesModel=gridSearch.fit(X_train,y_train['Species'])
BestCSpecies = SpeciesModel.best_params_['C']
BestGammaSpecies = SpeciesModel.best_params_['gamma']
BestModelSpeciesOVR = SVC(C = BestCSpecies, kernel = 'rbf', gamma = BestGammaSpecies,decision_function_shape='ovr')
SpeciesFinalModel = BestModelSpeciesOVR.fit(X_train,y_train['Species'])
y_pred = SpeciesFinalModel.predict(X_test)
HammingLossSpecies = hamming_loss(list(y_test['Species']), y_pred)
ExactMatchSpecies = accuracy_score(list(y_test['Species']), y_pred)
print('For the Species label : Best value of C =',BestCSpecies)
print('For the Species label : Best value of gamma =',BestGammaSpecies)
print('For the Species label : Hamming loss =',HammingLossSpecies)
print('For the Species label : Exact Match =',ExactMatchSpecies)

For the Species label : Best value of C = 1
For the Species label : Best value of gamma = 1.5
For the Species label : Hamming loss = 0.004219409282700422
For the Species label : Exact Match = 0.9957805907172996


In [23]:
standard=StandardScaler()
X_train_std = standard.fit_transform(X_train)
X_test_std = standard.transform(X_test)

In [26]:
modelSVM = SVC(kernel='rbf',decision_function_shape='ovr')
gridSearch = GridSearchCV(modelSVM, param_grid=param_grid, cv=10)
FamilyModel=gridSearch.fit(X_train_std,y_train['Family'])
BestCFamily = FamilyModel.best_params_['C']
BestGammaFamily = FamilyModel.best_params_['gamma']
BestModelFamilyOVR = SVC(C = BestCFamily, kernel = 'rbf', gamma = BestGammaFamily,decision_function_shape='ovr')
FamilyFinalModel = BestModelFamilyOVR.fit(X_train_std,y_train['Family'])
y_pred = FamilyFinalModel.predict(X_test_std)
HammingLossFamily = hamming_loss(list(y_test['Family']), y_pred)
ExactMatchFamily = accuracy_score(list(y_test['Family']), y_pred)
print("With Standardized Attributes:")
print('For the Family label : Best value of C =',BestCFamily)
print('For the Family label : Best value of gamma =',BestGammaFamily)
print('For the Family label : Hamming loss =',HammingLossFamily)
print('For the Family label : Exact Match =',ExactMatchFamily)

With Standardized Attributes:
For the Family label : Best value of C = 1
For the Family label : Best value of gamma = 0.1
For the Family label : Hamming loss = 0.0014064697609001407
For the Family label : Exact Match = 0.9985935302390999


In [27]:
modelSVM = SVC(kernel='rbf',decision_function_shape='ovr')
gridSearch = GridSearchCV(modelSVM, param_grid=param_grid, cv=10)
GenusModel = gridSearch.fit(X_train,y_train['Genus'])
BestCGenus = GenusModel.best_params_['C']
BestGammaGenus = GenusModel.best_params_['gamma']
BestModelGenusOVR = SVC(C = BestCGenus, kernel = 'rbf', gamma = BestGammaGenus,decision_function_shape='ovr')
GenusFinalModel = BestModelGenusOVR.fit(X_train,y_train['Genus'])
y_pred = GenusFinalModel.predict(X_test)
HammingLossGenus = hamming_loss(list(y_test['Family']), y_pred)
ExactMatchGenus = accuracy_score(list(y_test['Family']), y_pred)
print("With Standardized Attributes:")
print('For the Genus label : Best value of C =',BestCGenus)
print('For the Genus label : Best value of gamma =',BestGammaGenus)
print('For the Genus label : Hamming loss =',HammingLossGenus)
print('For the Genus label : Exact Match =',ExactMatchGenus)

With Standardized Attributes:
For the Genus label : Best value of C = 1
For the Genus label : Best value of gamma = 1.4
For the Genus label : Hamming loss = 1.0
For the Genus label : Exact Match = 0.0


In [29]:
modelSVM = SVC(kernel='rbf',decision_function_shape='ovr')
gridSearch = GridSearchCV(modelSVM, param_grid=param_grid, cv=10)
SpeciesModel=gridSearch.fit(X_train,y_train['Species'])
BestCSpecies = SpeciesModel.best_params_['C']
BestGammaSpecies = SpeciesModel.best_params_['gamma']
BestModelSpeciesOVR = SVC(C = BestCSpecies, kernel = 'rbf', gamma = BestGammaSpecies,decision_function_shape='ovr')
SpeciesFinalModel = BestModelSpeciesOVR.fit(X_train,y_train['Species'])
y_pred = SpeciesFinalModel.predict(X_test)
HammingLossSpecies = hamming_loss(list(y_test['Family']), y_pred)
ExactMatchSpecies = accuracy_score(list(y_test['Family']), y_pred)
print("With Standardized Attributes:")
print('For the Species label : Best value of C =',BestCSpecies)
print('For the Species label : Best value of gamma =',BestGammaSpecies)
print('For the Species label : Hamming loss =',HammingLossSpecies)
print('For the Species label : Exact Match =',ExactMatchSpecies)

With Standardized Attributes:
For the Species label : Best value of C = 1
For the Species label : Best value of gamma = 1.5
For the Species label : Hamming loss = 1.0
For the Species label : Exact Match = 0.0


### 1 (b) iii. L-1 Penalized SVM

In [30]:
param_grid = {'C':c}
SVC = LinearSVC(penalty='l1',dual = False)
gridSearch = GridSearchCV(SVC, param_grid=param_grid, cv=10)
FamilyModel = gridSearch.fit(X_train_std,y_train['Family'])
BestCFamily = FamilyModel.best_params_['C']
BestModelFamily = LinearSVC(penalty='l1',dual = False,C = BestCFamily)
FamilyFinalModel = BestModelFamily.fit(X_train_std,y_train['Family'])
y_pred = FamilyFinalModel.predict(X_test_std)
HammingLossFamily = hamming_loss(list(y_test['Family']), y_pred)
ExactMatchFamily = accuracy_score(list(y_test['Family']), y_pred)
print("With L1 Penalization:")
print('For the Family label : Best value of C =',BestCFamily)
print('For the Family label : Hamming loss =',HammingLossFamily)
print('For the Family label : Exact Match =',ExactMatchFamily)

With L1 Penalization:
For the Family label : Best value of C = 0.01
For the Family label : Hamming loss = 0.01969057665260197
For the Family label : Exact Match = 0.980309423347398


In [31]:
param_grid = {'C':c}
SVC = LinearSVC(penalty='l1',dual = False)
gridSearch = GridSearchCV(SVC, param_grid=param_grid, cv=10)
GenusModel = gridSearch.fit(X_train_std,y_train['Genus'])
BestCGenus = GenusModel.best_params_['C']
BestModelGenus = LinearSVC(penalty='l1',dual = False,C = BestCGenus)
GenusFinalModel = BestModelGenus.fit(X_train_std,y_train['Genus'])
y_pred = GenusFinalModel.predict(X_test_std)
HammingLossGenus = hamming_loss(list(y_test['Genus']), y_pred)
ExactMatchGenus = accuracy_score(list(y_test['Genus']), y_pred)
print("With L1 Penalization:")
print('For the Genus label : Best value of C =',BestCGenus)
print('For the Genus label : Hamming loss =',HammingLossGenus)
print('For the Genus label : Exact Match =',ExactMatchGenus)

For L1 Penalization:
For the Genus label : Best value of C = 0.01
For the Genus label : Hamming loss = 0.01969057665260197
For the Genus label : Exact Match = 0.980309423347398


In [33]:
param_grid = {'C':c}
SVC = LinearSVC(penalty='l1',dual = False)
gridSearch = GridSearchCV(SVC, param_grid=param_grid, cv=10)
SpeciesModel=gridSearch.fit(X_train_std,y_train['Species'])
BestCSpecies = SpeciesModel.best_params_['C']
BestModelSpecies = LinearSVC(penalty='l1',dual = False,C = BestCSpecies)
SpeciesFinalModel = BestModelSpecies.fit(X_train_std,y_train['Species'])
y_pred = SpeciesFinalModel.predict(X_test_std)
HammingLossSpecies = hamming_loss(list(y_test['Species']), y_pred)
ExactMatchSpecies = accuracy_score(list(y_test['Species']), y_pred)
print("With L1 Penalization:")
print('For the Species label : Best value of C =',BestCSpecies)
print('For the Species label : Hamming loss =',HammingLossSpecies)
print('For the Species label : Exact Match =',ExactMatchSpecies)

For L1 Penalization:
For the Species label : Best value of C = 1000.0
For the Species label : Hamming loss = 0.012658227848101266
For the Species label : Exact Match = 0.9873417721518988


### 1 (b) iv. SMOTE and L-1 penalized SVM

In [37]:
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=11)],
                                ['classifier', LinearSVC(penalty='l1',dual = False)]])
stratified_kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=11)
param_grid = {'classifier__C':c}
grid_search1 = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=stratified_kfold,
                           n_jobs=10)
grid_search1.fit(X_train_std, y_train['Family'])

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=11, shuffle=True),
             estimator=Pipeline(steps=[['smote', SMOTE(random_state=11)],
                                       ['classifier',
                                        LinearSVC(dual=False, penalty='l1')]]),
             n_jobs=10,
             param_grid={'classifier__C': [0.001, 0.01, 0.1, 1, 10.0, 100.0,
                                           1000.0, 10000.0]},
             scoring='roc_auc')

In [57]:
BestCFamily = FamilyModel.best_params_['C']
BestModelFamily = LinearSVC(penalty='l1',dual = False,C = BestCFamily)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_std, y_train['Family'])
FamilyFinalModel = BestModelFamily.fit(X_train_smote,y_train_smote)
y_pred = FamilyFinalModel.predict(X_test)
HammingLossFamily = hamming_loss(list(y_test['Family']), y_pred)
ExactMatchFamily = accuracy_score(list(y_test['Family']), y_pred)
print("With SMOTE and L-1 Penalization:")
print('For the Family label : Best value of C =',BestCFamily)
print('For the Family label : Hamming loss =',HammingLossFamily)
print('For the Family label : Exact Match =',ExactMatchFamily)

With SMOTE and L-1 Penalization:
For the Family label : Best value of C = 0.01
For the Family label : Hamming loss = 0.2250351617440225
For the Family label : Exact Match = 0.7749648382559775


In [58]:
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=11)],
                                ['classifier', LinearSVC(penalty='l1',dual = False)]])
stratified_kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=11)
param_grid = {'classifier__C':c}
grid_search1 = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=stratified_kfold,
                           n_jobs=10)
grid_search1.fit(X_train_std, y_train['Genus'])

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=11, shuffle=True),
             estimator=Pipeline(steps=[['smote', SMOTE(random_state=11)],
                                       ['classifier',
                                        LinearSVC(dual=False, penalty='l1')]]),
             n_jobs=10,
             param_grid={'classifier__C': [0.001, 0.01, 0.1, 1, 10.0, 100.0,
                                           1000.0, 10000.0]},
             scoring='roc_auc')

In [60]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_std, y_train['Genus'])
BestCGenus = GenusModel.best_params_['C']
BestModelGenus = LinearSVC(penalty='l1',dual = False,C = BestCGenus)
GenusFinalModel = BestModelGenus.fit(X_train_smote,y_train_smote)
y_pred = GenusFinalModel.predict(X_test)
HammingLossGenus = hamming_loss(list(y_test['Genus']), y_pred)
ExactMatchGenus = accuracy_score(list(y_test['Genus']), y_pred)
print("With SMOTE and L-1 Penalization:")
print('For the Genus label : Best value of C =',BestCGenus)
print('For the Genus label : Hamming loss =',HammingLossGenus)
print('For the Genus label : Exact Match =',ExactMatchGenus)

With SMOTE and L-1 Penalization:
For the Genus label : Best value of C = 0.01
For the Genus label : Hamming loss = 0.2250351617440225
For the Genus label : Exact Match = 0.7749648382559775


In [61]:
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=11)],
                                ['classifier', LinearSVC(penalty='l1',dual = False)]])
stratified_kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=11)
param_grid = {'classifier__C':c}
grid_search1 = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=stratified_kfold,
                           n_jobs=10)
grid_search1.fit(X_train_std, y_train['Species'])

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=11, shuffle=True),
             estimator=Pipeline(steps=[['smote', SMOTE(random_state=11)],
                                       ['classifier',
                                        LinearSVC(dual=False, penalty='l1')]]),
             n_jobs=10,
             param_grid={'classifier__C': [0.001, 0.01, 0.1, 1, 10.0, 100.0,
                                           1000.0, 10000.0]},
             scoring='roc_auc')

In [63]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_std, y_train['Species'])
BestCSpecies = SpeciesModel.best_params_['C']
BestModelSpecies = LinearSVC(penalty='l1',dual = False,C = BestCSpecies)
SpeciesFinalModel = BestModelSpecies.fit(X_train_smote,y_train_smote)
y_pred = SpeciesFinalModel.predict(X_test)
HammingLossSpecies = hamming_loss(list(y_test['Species']), y_pred)
ExactMatchSpecies = accuracy_score(list(y_test['Species']), y_pred)
print("With SMOTE and L-1 Penalization:")
print('For the Species label : Best value of C =',BestCGenus)
print('For the Species label : Hamming loss =',HammingLossGenus)
print('For the Species label : Exact Match =',ExactMatchGenus)

With SMOTE and L-1 Penalization:
For the Species label : Best value of C = 0.01
For the Species label : Hamming loss = 0.2250351617440225
For the Species label : Exact Match = 0.7749648382559775


**Results for SVM:**

For the Family label : Best value of C = 1

For the Family label : Best value of gamma = 1.4

For the Family label : Hamming loss = 0.0028129395218002813

For the Family label : Exact Match = 0.9971870604781997


---



For the Genus label : Best value of C = 1

For the Genus label : Best value of gamma = 1.4

For the Genus label : Hamming loss = 0.0028129395218002813

For the Genus label : Exact Match = 0.9971870604781997


---


For the Species label : Best value of C = 1

For the Species label : Best value of gamma = 1.5

For the Species label : Hamming loss = 0.004219409282700422

For the Species label : Exact Match = 0.9957805907172996

---

---

**Results for SVM with Standardized Attributes:**

With Standardized Attributes:

For the Family label : Best value of C = 1

For the Family label : Best value of gamma = 0.1

For the Family label : Hamming loss = 0.0014064697609001407

For the Family label : Exact Match = 0.9985935302390999

---

With Standardized Attributes:

For the Genus label : Best value of C = 1

For the Genus label : Best value of gamma = 1.4

For the Genus label : Hamming loss = 1.0

For the Genus label : Exact Match = 0.0

---


With Standardized Attributes:

For the Species label : Best value of C = 1

For the Species label : Best value of gamma = 1.5

For the Species label : Hamming loss = 1.0

For the Species label : Exact Match = 0.0

---

---

**Results with L-1 Penalized SVM:**

With L1 Penalization:

For the Family label : Best value of C = 0.01

For the Family label : Hamming loss = 0.01969057665260197

For the Family label : Exact Match = 0.980309423347398

---

For L1 Penalization:

For the Genus label : Best value of C = 0.01

For the Genus label : Hamming loss = 0.01969057665260197

For the Genus label : Exact Match = 0.980309423347398

---

For L1 Penalization:

For the Species label : Best value of C = 1000.0

For the Species label : Hamming loss = 0.012658227848101266

For the Species label : Exact Match = 0.9873417721518988

---

---

**Results with SMOTE and L-1 Penalized SVM:**

With SMOTE and L-1 Penalization:

For the Family label : Best value of C = 0.01

For the Family label : Hamming loss = 0.2250351617440225

For the Family label : Exact Match = 0.7749648382559775

---

With SMOTE and L-1 Penalization:

For the Genus label : Best value of C = 0.01

For the Genus label : Hamming loss = 0.2250351617440225

For the Genus label : Exact Match = 0.7749648382559775

---

With SMOTE and L-1 Penalization:

For the Species label : Best value of C = 0.01

For the Species label : Hamming loss = 0.2250351617440225

For the Species label : Exact Match = 0.7749648382559775

---

From the above results we can infer that the exact match evaluation for Gaussian Kernal with both Raw and Standardized data is the same, and the Hamming score/loss for Gaussian Kernal with Raw data is the lowest. As a result, the Gassian Kernal classifier outperforms L1 and Smote.

### 2 (a) K-means clustering

In [5]:
LH = []
DH = []

In [6]:
for i in range(1,51):
    silh_scores = []
    for k in range(2, 51):
        kmc = KMeans(n_clusters=k, random_state=i)
        Z = kmc.fit_predict(X)
        silh_scores.append(silhouette_score(X, Z))
    bestK = silh_scores.index(max(silh_scores)) + 2  
    print('Iteration={} || k={}'.format(i,bestK))
    kmc = KMeans(n_clusters=bestK, random_state=i)
    Z = kmc.fit_predict(X)
    mismatch = 0
    y['pred'] = Z
    tl = {}
    for j in range(bestK):
        dict1 = {}
        match = y[y['pred'] == j]
        for col in y.columns[:-1]:
            dict1[col] = Counter(match[col]).most_common(1)[0][0]
            mismatch += sum(match[col] != dict1[col])
        tl[j] = dict1
    majority = pd.DataFrame(tl).T
    print("Majority label= \n",majority)
    HD = mismatch / y.shape[0]    
    HL = mismatch / (y.shape[0] * 3)
    LH.append(HL)
    DH.append(HD)
    print("Hamming loss = ",HL)
    print("Hamming Distance =",HD)
    print("\n")

Iteration=1 || k=3
Majority label= 
             Family      Genus                 Species
0    Dendrobatidae   Ameerega      Ameeregatrivittata
1  Leptodactylidae  Adenomera  AdenomeraHylaedactylus
2  Leptodactylidae  Adenomera          AdenomeraAndre
Hamming loss =  0.13052039381153305
Hamming Distance = 0.39156118143459917


Iteration=2 || k=3
Majority label= 
             Family      Genus                 Species
0  Leptodactylidae  Adenomera  AdenomeraHylaedactylus
1  Leptodactylidae  Adenomera          AdenomeraAndre
2    Dendrobatidae   Ameerega      Ameeregatrivittata
Hamming loss =  0.13052039381153305
Hamming Distance = 0.39156118143459917


Iteration=3 || k=3
Majority label= 
             Family      Genus                 Species
0  Leptodactylidae  Adenomera  AdenomeraHylaedactylus
1    Dendrobatidae   Ameerega      Ameeregatrivittata
2  Leptodactylidae  Adenomera          AdenomeraAndre
Hamming loss =  0.13052039381153305
Hamming Distance = 0.39156118143459917


Iteration=

Best value of K is 3

In [8]:
print("Average Hamming Distance =", np.average(DH))
print("Average Hamming Score =", np.average(1 - np.array(LH)))
print("Average Hamming Loss =", np.average(LH))
print("STD of Hamming Distance =", np.std(DH))
print("STD of Hamming Loss =", np.std(LH))

Average Hamming Distance = 0.3915611814345992
Average Hamming Score = 0.8694796061884671
Average Hamming Loss = 0.13052039381153305
STD of Hamming Distance = 5.551115123125783e-17
STD of Hamming Loss = 0.0
